In [1]:
import keras
import model as m
from skimage import io
from skimage.draw import circle
import numpy as np
import xml.etree.ElementTree as ET

TILE_X = 200
TILE_Y = 150

Using TensorFlow backend.


In [ ]:
image = io.imread("images/13.tif")
io.imshow(image)

In [ ]:
cut = image[:200,:150]

mask = np.zeros(cut.shape)

io.imshow_collection([cut,mask])
# io.imshow(mask)

In [ ]:
tree = ET.parse('images/CellCounter_13.xml')
root = tree.getroot()

for marker in root.iter("Marker"):
    y = int(marker[0].text)
    x = int(marker[1].text)
    if(x < 200 and y < 150):
        rr,cc = circle(x,y,6, mask.shape)
        mask[rr,cc] = 1

io.imshow_collection([cut,mask])

In [ ]:
#get 10 cuts and masks 
dataset = ["6","7","8","13"]
imgs_train = []
imgs_mask = []

for i in dataset:
    image = io.imread("images/" + i + ".tif")
    cut = image[:TILE_X,:TILE_Y]
    mask = np.zeros(cut.shape)
    
    #add circles 
    tree = ET.parse('images/CellCounter_' + i + '.xml')
    root = tree.getroot()

    for marker in root.iter("Marker"):
        y = int(marker[0].text)
        x = int(marker[1].text)
        if(x < TILE_X and y < TILE_Y):
            rr,cc = circle(x,y,6, mask.shape)
            mask[rr,cc] = 1
    imgs_train.append(cut)
    imgs_mask.append(mask)
    
io.imshow_collection([imgs_train[0],imgs_mask[0]])

In [ ]:
model = m.unet(input_size = (TILE_X,TILE_Y,1))

In [ ]:
model.fit(imgs_train, imgs_mask, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True)